In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# import utilities
import sys 
import json
import importlib
sys.path.append('../')
reset = importlib.import_module("utilities.reset")
reset_pipeline = reset.reset_pipeline

# load secrets from a .env file using python-dotenv
from dotenv import load_dotenv
import os
load_dotenv("../../.env")
MY_API_KEY = os.getenv('MY_API_KEY')
MY_API_URL = os.getenv('MY_API_URL')

# import krixik and initialize it with your personal secrets
from krixik import krixik
krixik.init(api_key = MY_API_KEY, 
            api_url = MY_API_URL)

SUCCESS: You are now authenticated.


In [2]:
import yaml

file_path = "../mkdocs.yml"
with open(file_path, "r") as file:
    mkdocks_toc = yaml.safe_load(file)

pipeline_examples_docs = mkdocks_toc["nav"][3]["Pipeline examples"]
system_docs = mkdocks_toc["nav"][5]["System"][0]["methods"]
modules_docs = mkdocks_toc["nav"][4]["Modules"][0]["currently available"]
builder_docs = mkdocks_toc["nav"][6]["Pipeline builder"]


# collect paths to example docs
examples_mds = []
for item in pipeline_examples_docs:
    item_values = list(item.values())[0]
    for subitem in item_values:
        docpath = list(subitem.values())[0]
        examples_mds.append(docpath)
        
# collect paths to system docs
system_mds = []
for item in system_docs:
    docpath = list(item.values())[0]
    system_mds.append(docpath)
    
# collect paths to modules docs
modules_mds = []
for item in modules_docs:
    docpath = list(item.values())[0]
    modules_mds.append(docpath)
    
# collect paths to builder docs
builder_mds = []
for item in builder_docs[0]["Overview"]:
    docpath = list(item.values())[0]
    builder_mds.append(docpath)
for item in builder_docs[1]["Modules - advanced details"]:
    docpath = list(item.values())[0]
    builder_mds.append(docpath)
for item in builder_docs[2]["Pipelines - advanced details"]:
    docpath = list(item.values())[0]
    builder_mds.append(docpath)
    
# merge all mds
all_mds = examples_mds + system_mds + modules_mds + builder_mds

In [3]:
import subprocess


def convert_notebook_to_markdown(docpath: str) -> None:
    command = [
        "jupyter",
        "nbconvert",
        f"{docpath}",
        "--TagRemovePreprocessor.enabled=True",
        "--TagRemovePreprocessor.remove_cell_tags=['remove_cell']",
        "--TagRemovePreprocessor.remove_all_outputs_tags=['remove_output']",
        "--to",
        "markdown"
    ]

    # Run the command
    process = subprocess.run(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

    # Check the return code
    if process.returncode != 0:
        # Print the error message
        print("Error:", process.stderr.decode().strip())


In [4]:
for i in range(len(all_mds)):
    docpath = "../docs/" + all_mds[i].replace(".md",".ipynb")
    convert_notebook_to_markdown(docpath)

Error: [NbConvertApp] WARNING | pattern '../docs/modules/overview.ipynb' matched no files
Error: [NbConvertApp] WARNING | pattern '../docs/pipeline_builder/introduction.ipynb' matched no files
Error: [NbConvertApp] WARNING | pattern '../docs/pipeline_builder/existing_modules.ipynb' matched no files
Error: [NbConvertApp] WARNING | pattern '../docs/pipeline_builder/create_module.ipynb' matched no files
Error: [NbConvertApp] WARNING | pattern '../docs/pipeline_builder/view_module_config.ipynb' matched no files
Error: [NbConvertApp] WARNING | pattern '../docs/pipeline_builder/first_pipeline.ipynb' matched no files
Error: [NbConvertApp] WARNING | pattern '../docs/pipeline_builder/testing_pipeline.ipynb' matched no files
Error: [NbConvertApp] WARNING | pattern '../docs/pipeline_builder/useful_pipeline_properties.ipynb' matched no files
Error: [NbConvertApp] WARNING | pattern '../docs/pipeline_builder/save_load.ipynb' matched no files
Error: [NbConvertApp] WARNING | pattern '../docs/advanced_